##### Copyright 2021 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Isolated XEB

This notebook demonstrates how to use the functionality in `cirq.experiments` to run Isolated XEB end-to-end. "Isolated" means we do one pair of qubits at a time.

In [ ]:
import cirq
import numpy as np

## Set up Random Circuits

We create a library of 20 random, two-qubit `circuits` using the sqrt(ISWAP) gate on the two qubits we've chosen.

In [ ]:
from cirq.experiments import random_quantum_circuit_generation as rqcg

circuits = rqcg.generate_library_of_2q_circuits(
    n_library_circuits=20,
    two_qubit_gate=cirq.ISWAP**0.5,
    q0=cirq.GridQubit(4,4),
    q1=cirq.GridQubit(4,5),
)
print(len(circuits))

In [ ]:
# We will truncate to these lengths
cycle_depths = np.arange(3, 100, 20)
cycle_depths

### Set up a `Sampler`.

For demonstration, we'll use a density matrix simulator to sample noisy samples. However, input a `device_name` (and have an authenticated Google Cloud project name set as your `GOOGLE_CLOUD_PROJECT` environment variable) to run on a real device.

In [ ]:
device_name = None  # change me!

if device_name is None:
    sampler = cirq.DensityMatrixSimulator(noise=cirq.depolarize(5e-3))
else:
    import cirq.google as cg
    sampler = cg.get_engine_sampler(device_name, gate_set_name='sqrt_iswap')
    device = cg.get_engine_device(device_name)

    import cirq.contrib.routing as ccr
    graph = ccr.xmon_device_to_graph(device)
    pos = {q: (q.row, q.col) for q in graph.nodes}
    nx.draw_networkx(graph, pos=pos)

## Take Data

In [ ]:
from cirq.experiments.xeb_sampling import sample_2q_xeb_circuits
sampled_df = sample_2q_xeb_circuits(
    sampler=sampler,
    circuits=circuits,
    cycle_depths=cycle_depths,
)
sampled_df

## Benchmark fidelities

In [ ]:
from cirq.experiments.xeb_fitting import benchmark_2q_xeb_fidelities
fids = benchmark_2q_xeb_fidelities(
    sampled_df=sampled_df,
    circuits=circuits,
    cycle_depths=cycle_depths,
)
fids

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

# Exponential reference
xx = np.linspace(0, fids['cycle_depth'].max())
plt.plot(xx, (1-5e-3)**(4*xx), label=r'Exponential Reference')

def _p(fids):
    plt.plot(fids['cycle_depth'], fids['fidelity'], 'o-', label=fids.name)

fids.name = 'Sampled'
_p(fids)

plt.ylabel('Circuit fidelity')
plt.xlabel('Cycle Depth $d$')
plt.legend(loc='best')